<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/08_RAG.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 08. RAG

## Overview  
In this exercise, we will explore Retrieval-Augmented Generation (RAG) using the Solar framework. RAG combines retrieval-based techniques with generative models to improve the relevance and accuracy of generated responses by leveraging external knowledge sources. This notebook will guide you through implementing RAG and demonstrating its benefits in enhancing model outputs.

## Purpose of the Exercise
The purpose of this exercise is to integrate Retrieval-Augmented Generation into the Solar framework. By the end of this tutorial, users will understand how to use RAG to access external information and generate more informed and contextually accurate responses, thereby improving the performance and reliability of the language model.



## RAG: Retrieval Augmented Generation.
- Large language models (LLMs) have a limited context size.
- TLDR
- Not all context is relevant to a given question
- Query → Retrieve (Search) →  Results →  (LLM) →  Answer
- RAG is a method to combine LLM with Retrieval: Retrieval Augmented Generation



In [ ]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25 python-dotenv tokenizers

In [2]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


In [16]:
from langchain_upstage import UpstageLayoutAnalysisLoader


layzer = UpstageLayoutAnalysisLoader(
    "pdfs/kim-tse-2008.pdf", use_ocr=False, output_type="html"
)
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [17]:
from IPython.display import display, HTML

display(HTML(docs[0].page_content[:1000]))

In [20]:
from langchain_upstage import UpstageDocumentParseLoader


layzer = UpstageDocumentParseLoader(
    "pdfs/kim-tse-2008.pdf", ocr=False, output_format="html"
)
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [21]:
from IPython.display import display, HTML

display(HTML(docs[0].page_content[:1000]))

In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context.
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [23]:
chain.invoke({"question": "What is bug classficiation?", "Context": docs})

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 32768 tokens. However, your messages resulted in 52259 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [26]:
! pip3 install -qU langchain_community

In [27]:
from langchain_community.retrievers import BM25Retriever
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print(len(splits))



132


In [28]:
from transformers import AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

solar_tokenizer = AutoTokenizer.from_pretrained("upstage/solar-pro-preview-instruct")

token_splitter = TokenTextSplitter.from_huggingface_tokenizer(
    solar_tokenizer, chunk_size=250, chunk_overlap=100
)

splits = token_splitter.split_documents(docs)
print(len(splits))

tokenizer_config.json:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

177


In [32]:
retriever = BM25Retriever.from_documents(splits)
retriever.invoke("What is bug classficiation?")

[Document(metadata={'total_pages': 16, 'coordinates': [[{'x': 0.0396, 'y': 0.0561}, {'x': 0.5623, 'y': 0.0561}, {'x': 0.5623, 'y': 0.0688}, {'x': 0.0396, 'y': 0.0688}], [{'x': 0.8614, 'y': 0.0571}, {'x': 0.8826, 'y': 0.0571}, {'x': 0.8826, 'y': 0.0675}, {'x': 0.8614, 'y': 0.0675}], [{'x': 0.1885, 'y': 0.0897}, {'x': 0.7413, 'y': 0.0897}, {'x': 0.7413, 'y': 0.1589}, {'x': 0.1885, 'y': 0.1589}], [{'x': 0.1016, 'y': 0.1696}, {'x': 0.824, 'y': 0.1696}, {'x': 0.824, 'y': 0.1877}, {'x': 0.1016, 'y': 0.1877}], [{'x': 0.0787, 'y': 0.2091}, {'x': 0.8515, 'y': 0.2091}, {'x': 0.8515, 'y': 0.356}, {'x': 0.0787, 'y': 0.356}], [{'x': 0.0785, 'y': 0.3693}, {'x': 0.8468, 'y': 0.3693}, {'x': 0.8468, 'y': 0.3972}, {'x': 0.0785, 'y': 0.3972}], [{'x': 0.4545, 'y': 0.3997}, {'x': 0.4711, 'y': 0.3997}, {'x': 0.4711, 'y': 0.417}, {'x': 0.4545, 'y': 0.417}], [{'x': 0.0391, 'y': 0.4394}, {'x': 0.1883, 'y': 0.4394}, {'x': 0.1883, 'y': 0.4553}, {'x': 0.0391, 'y': 0.4553}], [{'x': 0.04, 'y': 0.4603}, {'x': 0.4557

In [30]:
query = "What is bug classficiation?"
context_docs = retriever.invoke(query)
chain.invoke({"question": query, "Context": context_docs})

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 32768 tokens. However, your messages resulted in 90338 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [33]:
query = "How do find the bug?"
context_docs = retriever.invoke(query)
chain.invoke({"question": query, "Context": context_docs})

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 32768 tokens. However, your messages resulted in 90335 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

# Excercise
It seems keyword search is not the best for LLM queries. What are some alternatives?